<a href="https://colab.research.google.com/github/GinuraAdikari/InsightHive/blob/Sentiment_Analysis/Model_ABSA_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer
import pandas as pd
import numpy as np

ABSA = pd.read_csv("ABSA_dataset.csv")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [2]:
ABSA.head()

,tokens,aspect_mask,sentiment
0,"['one', 'best', 'game', 'music', 'soundtrack',...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, ...",1
1,"['best', 'purchase', 'ever', 'bought', 'ex', '...","[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, ...",1
2,"['book', 'slow', 'weak', 'one', 'best', '##st'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, ...",0
3,"['must', '##rea', '##d', 'every', 'southern', ...","[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ...",1
4,"['horrible', 'watch', 'napoleon', 'want', 'fun...","[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0


In [ ]:
#------
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def convert_tokens_to_ids(row):
    """Converts tokenized words to BERT token IDs."""
    input_ids = tokenizer.convert_tokens_to_ids(eval(row["tokens"]))
    return input_ids

# Apply conversion
ABSA_data["input_ids"] = ABSA_data.apply(convert_tokens_to_ids, axis=1)

# Convert to PyTorch tensors for training
input_ids = torch.tensor(ABSA_data["input_ids"].tolist(), dtype=torch.long)
aspect_masks = torch.tensor(ABSA_data["aspect_mask"].tolist(), dtype=torch.long)
labels = torch.tensor(ABSA_data["sentiment"].tolist(), dtype=torch.long)

print("Tokens converted to input IDs for model training!")

In [ ]:
#------
from torch.nn.utils.rnn import pad_sequence

max_length = 128  # Set a fixed length for input sequences

# Apply padding
input_ids = pad_sequence([torch.tensor(x) for x in ABSA_data["input_ids"]], batch_first=True, padding_value=0)
aspect_masks = pad_sequence([torch.tensor(x) for x in ABSA_data["aspect_mask"]], batch_first=True, padding_value=0)
labels = torch.tensor(ABSA_data["sentiment"].tolist(), dtype=torch.long)

In [3]:
from sklearn.model_selection import train_test_split

# train (80%) and test (20%)
train_df, test_df = train_test_split(ABSA, test_size=0.2, random_state=42, stratify=ABSA["sentiment"])

print("Train and test datasets created!")

Train and test datasets created!


# Model input processing

In [4]:
def convert_example_to_features(example, max_seq_length=128):
    tokens = eval(example["tokens"])
    aspect_mask = eval(example["aspect_mask"])

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Truncate if necessary
    if len(input_ids) > max_seq_length:
        input_ids = input_ids[:max_seq_length]
        aspect_mask = aspect_mask[:max_seq_length]

    # Pad shorter sequences
    pad_length = max_seq_length - len(input_ids)
    input_ids += [0] * pad_length  # PAD token
    aspect_mask += [0] * pad_length  # Pad aspect mask

    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "input_mask": torch.tensor([1] * len(input_ids), dtype=torch.long),
        "aspect_mask": torch.tensor(aspect_mask, dtype=torch.long)
    }

# Apply feature conversion on the train split
train_features = train_df.apply(convert_example_to_features, axis=1)

# Extract training tensors
train_input_ids = torch.stack([f["input_ids"] for f in train_features])
train_input_mask = torch.stack([f["input_mask"] for f in train_features])
train_aspect_mask = torch.stack([f["aspect_mask"] for f in train_features])
train_labels = torch.tensor(train_df["sentiment"].values, dtype=torch.long)

# Build training DataLoader
train_data = TensorDataset(train_input_ids, train_input_mask, train_aspect_mask, train_labels)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

# Apply feature conversion on the test split
test_features = test_df.apply(convert_example_to_features, axis=1)

# Extract test tensors
test_input_ids = torch.stack([f["input_ids"] for f in test_features])
test_input_mask = torch.stack([f["input_mask"] for f in test_features])
test_aspect_mask = torch.stack([f["aspect_mask"] for f in test_features])
test_labels = torch.tensor(test_df["sentiment"].values, dtype=torch.long)

# Build test DataLoader
test_data = TensorDataset(test_input_ids, test_input_mask, test_aspect_mask, test_labels)
test_loader = DataLoader(test_data, batch_size=16)

print("Train and test datasets created and DataLoaders are ready!")

train_df.to_csv("absa_train_dataset.csv", index=False)
test_df.to_csv("absa_test_dataset.csv", index=False)

Train and test datasets created and DataLoaders are ready!


# Aspect-Aware BERT Model

In [5]:
import torch
import torch.nn as nn
from transformers import BertModel

class BertForABSA(nn.Module):
    def __init__(self, bert_model=None, num_labels=2):
        super(BertForABSA, self).__init__()

        # Load BERT model
        self.bert = bert_model if bert_model else BertModel.from_pretrained("bert-base-uncased")

        # Aspect attention mechanism
        self.aspect_attention = nn.Linear(self.bert.config.hidden_size, 1)

        # Sentiment classifier
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, aspect_mask, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        last_hidden_state = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)
        pooled_output = outputs.pooler_output  # Shape: (batch_size, hidden_size)

        # Compute attention scores for each token
        aspect_weights = torch.softmax(self.aspect_attention(last_hidden_state), dim=1)  # Shape: (batch_size, seq_len, 1)

        # Apply aspect mask at token level
        aspect_mask = aspect_mask.unsqueeze(-1).float()  # Expand to (batch_size, seq_len, 1)
        aspect_pooled = torch.sum(last_hidden_state * aspect_weights * aspect_mask, dim=1)  # Sum over sequence length

        logits = self.classifier(aspect_pooled)  # Final sentiment prediction
        return logits

# Handling Class Imbalance

In [6]:
from sklearn.utils.class_weight import compute_class_weight

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
labels = torch.tensor(train_df["sentiment"].values, dtype=torch.long)

# Compute class weights
labels_np = labels.cpu().numpy()
class_weights = compute_class_weight("balanced", classes=np.unique(labels_np), y=labels_np)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

In [7]:
#------
import torch.optim as optim

# Load best hyperparameters
best_params = {"learning_rate": 2e-5, "batch_size": 16, "weight_decay": 0.01}

# Load balanced dataset
train_loader = DataLoader(balanced_train_data, batch_size=best_params["batch_size"], shuffle=True)

# Load model
model = BertForABSA.from_pretrained("bert-base-uncased", num_labels=3)
optimizer = optim.AdamW(model.parameters(), lr=best_params["learning_rate"], weight_decay=best_params["weight_decay"])
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

# Training loop
for epoch in range(5):
    model.train()
    for batch in train_loader:
        input_ids, input_mask, aspect_mask, labels = batch

        optimizer.zero_grad()
        logits = model(input_ids, aspect_mask, input_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")


NameError: name 'balanced_train_data' is not defined

# Train the Model

In [8]:
import torch.optim as optim
from transformers import get_linear_schedule_with_warmup

# Load model
bert_model = BertModel.from_pretrained("bert-base-uncased")
model = BertForABSA(bert_model=bert_model, num_labels=2)
model.to(device)

num_epochs = 3

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

model.train()

# Training loop
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids, attention_mask, aspect_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids, aspect_mask, attention_mask=attention_mask)
        logits = outputs
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f"Epoch {epoch+1} Loss: {loss.item()}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1 Loss: 0.6876569986343384
Epoch 2 Loss: 0.6936724781990051
Epoch 3 Loss: 0.6936954259872437


In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer

# Load preprocessed dataset
df = pd.read_csv("absa_dataset.csv")

# Convert to tensors
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
input_ids = [tokenizer.convert_tokens_to_ids(eval(row["tokens"])) for _, row in df.iterrows()]
aspect_masks = [eval(row["aspect_mask"]) for _, row in df.iterrows()]
labels = df["sentiment"].values

input_ids = torch.tensor(input_ids, dtype=torch.long)
aspect_masks = torch.tensor(aspect_masks, dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.long)

# Create DataLoader for training
train_data = TensorDataset(input_ids, aspect_masks, labels)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class BertForABSA(nn.Module):
    def __init__(self, num_labels=3):
        super(BertForABSA, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.aspect_attention = nn.Linear(self.bert.config.hidden_size, 1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, aspect_mask, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        pooled_output = outputs.pooler_output  # Correct way to access pooled output

        aspect_weights = torch.softmax(self.aspect_attention(pooled_output), dim=1)
        aspect_pooled = pooled_output * aspect_weights * aspect_mask.unsqueeze(-1)

        logits = self.classifier(aspect_pooled)
        return {"logits": logits}  # Wrap logits in a dictionary

In [ ]:
import torch
from transformers import BertTokenizer
from modeling_new import BertForABSA
import pandas as pd

# Load model & tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForABSA.from_pretrained("saved_model/absa_model.bin")
model.eval()

# Load test data
df = pd.read_csv("absa_dataset.csv")

input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(eval(row["tokens"])) for _, row in df.iterrows()], dtype=torch.long)
aspect_masks = torch.tensor([eval(row["aspect_mask"]) for _, row in df.iterrows()], dtype=torch.long)

with torch.no_grad():
    logits = model(input_ids, aspect_masks)
    predictions = torch.argmax(logits, dim=1)

df["Predicted_Sentiment"] = predictions.numpy()
df.to_csv("absa_predictions.csv", index=False)


# Evaluate the Model

In [ ]:
import torch
from sklearn.metrics import classification_report

# Load trained model
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Initialize BertForABSA with 2 sentiment labels (Binary Classification)
model = BertForABSA(bert_model=bert_model)
model.eval()

all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids, input_mask, aspect_mask, labels = batch
        outputs = model(input_ids, aspect_mask, input_mask)
        logits = outputs["logits"]  # Extract logits
        predictions = torch.argmax(logits, dim=1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print evaluation metrics
print("Model Performance on Test Set:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"]))

IndexError: too many indices for tensor of dimension 2

# Hyperparameter Optimization

In [ ]:
import torch
import torch.optim as optim
from sklearn.model_selection import ParameterGrid

# Define possible hyperparameters
param_grid = {
    "learning_rate": [2e-5, 3e-5, 5e-5],
    "batch_size": [8, 16],
    "weight_decay": [0.01, 0.001]
}

# Create all possible combinations
param_combinations = list(ParameterGrid(param_grid))

best_model = None
best_val_loss = float("inf")

for params in param_combinations:
    print(f"\nTraining with params: {params}")

    # Load model
    model = BertForABSA.from_pretrained("bert-base-uncased", num_labels=3)
    optimizer = optim.AdamW(model.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])
    criterion = torch.nn.CrossEntropyLoss()

    train_loader = DataLoader(train_data, batch_size=params["batch_size"], shuffle=True)
    val_loader = DataLoader(val_data, batch_size=params["batch_size"])

    for epoch in range(3):
        model.train()
        for batch in train_loader:
            input_ids, input_mask, aspect_mask, labels = batch
            optimizer.zero_grad()
            logits = model(input_ids, aspect_mask, input_mask)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

    # Validate the model
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, aspect_mask, labels = batch
            logits = model(input_ids, aspect_mask)
            loss = criterion(logits, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss}")

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = model

# Save the best model
best_model.save_pretrained("best_absa_model")
print("Best model saved!")


# Model Evaluation: Precision, Recall, F1-Score

In [9]:
import torch
import pandas as pd
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [10]:
# Helper Function: Load Data

def load_data(filename, max_length=None):

    df = pd.read_csv(filename)

    # Compute maximum sequence length from tokens (if not provided)
    if max_length is None:
        max_length = min(512, max(len(eval(row["tokens"])) for _, row in df.iterrows()))

    # Convert tokens and aspect masks into lists of IDs (truncated to max_length)
    input_ids_list = [tokenizer.convert_tokens_to_ids(eval(row["tokens"]))[:max_length]
                      for _, row in df.iterrows()]
    aspect_masks_list = [eval(row["aspect_mask"])[:max_length]
                         for _, row in df.iterrows()]

    # Pad sequences to max_length
    input_ids = pad_sequence([torch.tensor(seq) for seq in input_ids_list],
                             batch_first=True, padding_value=tokenizer.pad_token_id)
    aspect_masks = pad_sequence([torch.tensor(mask) for mask in aspect_masks_list],
                                batch_first=True, padding_value=0)

    # Create attention masks: 1 for real tokens, 0 for padding
    attention_masks = (input_ids != tokenizer.pad_token_id).long()

    # Convert sentiment labels to tensor
    labels = torch.tensor(df["sentiment"].values, dtype=torch.long)

    # Return a TensorDataset, the DataFrame, and max_length used
    dataset = TensorDataset(input_ids, attention_masks, aspect_masks, labels)
    return dataset, df, max_length

In [11]:
# Helper Function: Evaluate Model

def evaluate_model(model, dataloader):
    """
    Evaluates the model on the given dataloader and returns predictions and true labels.
    """
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, aspect_mask, labels = [b.to(device) for b in batch]
            # Forward pass: make sure to pass the attention_mask
            outputs = model(input_ids, aspect_mask, attention_mask=attention_mask)
            # Assuming your model returns logits directly
            logits = outputs
            predictions = torch.argmax(logits, dim=1)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_preds, all_labels

In [12]:
# Load Train and Test Data

train_dataset, train_df, max_length_train = load_data("absa_train_dataset.csv")
# Use the same max_length for test data to ensure consistent tensor sizes.
test_dataset, test_df, _ = load_data("absa_test_dataset.csv", max_length=max_length_train)

# Create DataLoaders (set shuffle=False during evaluation)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16)

In [13]:
# Initialize and Load Your Model

# Ensure your BertForABSA class is defined (see previous examples)
from transformers import BertModel  # and your custom BertForABSA

# Load pre-trained BERT and initialize your ABSA model.
bert_model = BertModel.from_pretrained("bert-base-uncased")
model = BertForABSA(bert_model=bert_model, num_labels=2)  # for binary sentiment
model.to(device)

BertForABSA(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [14]:
# Evaluate on Training Data

train_preds, train_labels = evaluate_model(model, train_loader)
print("Training Performance:")
print(classification_report(train_labels, train_preds, target_names=["Negative", "Positive"]))


# Evaluate on Test Data

test_preds, test_labels = evaluate_model(model, test_loader)
print("Test Performance:")
print(classification_report(test_labels, test_preds, target_names=["Negative", "Positive"]))

Training Performance:
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       330
    Positive       0.68      1.00      0.81       718

    accuracy                           0.68      1048
   macro avg       0.34      0.50      0.41      1048
weighted avg       0.47      0.68      0.56      1048

Test Performance:
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        82
    Positive       0.69      0.99      0.81       180

    accuracy                           0.68       262
   macro avg       0.34      0.50      0.41       262
weighted avg       0.47      0.68      0.56       262

